## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update   `REGION`, and `ARTIFACT_STORE`  with the settings reflecting your lab environment. 

- `REGION` - the compute region for Vertex AI Training and Prediction
- `ARTIFACT_STORE` - A GCS bucket in the created in the same region.

Containerized Training Cloud Serving
- `Phase 1` - (Before ML Pipeline)

In [30]:
import os
import time

from google.cloud import aiplatform

In [31]:
REGION = "us-east1"

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

ARTIFACT_STORE = f"gs://{PROJECT_ID}-beer-artifact-store"

DATA_ROOT = f"{ARTIFACT_STORE}/data"
JOB_DIR_ROOT = f"{ARTIFACT_STORE}/jobs"
TRAINING_FILE_PATH = f"{DATA_ROOT}/train.parquet"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/valid.parquet"
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [32]:
os.environ["JOB_DIR_ROOT"] = JOB_DIR_ROOT
os.environ["TRAINING_FILE_PATH"] = TRAINING_FILE_PATH
os.environ["VALIDATION_FILE_PATH"] = VALIDATION_FILE_PATH
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["REGION"] = REGION

In [33]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/


### Package the script into a docker image.

Notice that we are installing specific versions of `scikit-learn` and `pandas` in the training image. This is done to make sure that the training runtime in the training container is aligned with the serving runtime in the serving container. 

Make sure to update the URI for the base image so that it points to your project's **Container Registry**.

In [34]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting training_app/Dockerfile


In [35]:
IMAGE_NAME = "trainer_image"
IMAGE_TAG = "latest"
IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{IMAGE_TAG}"

os.environ["IMAGE_URI"] = IMAGE_URI

In [36]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 3 file(s) totalling 12.2 KiB before compression.
Uploading tarball of [training_app] to [gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654231732.212619-706f54b26dff490f83816fbb858057a2.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-04-5e165f533cac/locations/global/builds/6f4f124f-d204-4cc1-aef9-7b0bd4e5efe2].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/6f4f124f-d204-4cc1-aef9-7b0bd4e5efe2?project=547029906128].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6f4f124f-d204-4cc1-aef9-7b0bd4e5efe2"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654231732.212619-706f54b26dff490f83816fbb858057a2.tgz#1654231732422676
Copying gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654231732.212619-706f54b26dff490f83816fbb858057a2.tgz#1654231732422676...
/ [1 files][  2.7 KiB/  2.7 KiB]                   

## Submit an Vertex AI hyperparameter tuning job

In [37]:
TIMESTAMP = time.strftime("%Y%m%d_%H%M%S")
JOB_NAME = f"beer_recom_tuning_{TIMESTAMP}"
JOB_DIR = f"{JOB_DIR_ROOT}/{JOB_NAME}"

os.environ["JOB_NAME"] = JOB_NAME
os.environ["JOB_DIR"] = JOB_DIR

In [38]:
%%bash

CONFIG_YAML=config.yaml

gcloud ai hp-tuning-jobs create \
    --region=$REGION \
    --display-name=$JOB_NAME \
    --config=$CONFIG_YAML \
    --max-trial-count=5 \
    --parallel-trial-count=5

echo "JOB_NAME: $JOB_NAME"

JOB_NAME: beer_recom_tuning_20220603_045016


Using endpoint [https://us-east1-aiplatform.googleapis.com/]
Hyperparameter tuning job [2329788173443399680] submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai hp-tuning-jobs describe 2329788173443399680 --region=us-east1

Job State: JOB_STATE_PENDING


In [41]:
!gcloud ai hp-tuning-jobs describe 2329788173443399680 --region=us-east1

Using endpoint [https://us-east1-aiplatform.googleapis.com/]
createTime: '2022-06-03T04:50:17.775809Z'
displayName: beer_recom_tuning_20220603_045016
maxTrialCount: 5
name: projects/547029906128/locations/us-east1/hyperparameterTuningJobs/2329788173443399680
parallelTrialCount: 5
startTime: '2022-06-03T04:50:24Z'
state: JOB_STATE_RUNNING
studySpec:
  metrics:
  - goal: MAXIMIZE
    metricId: map_at_10
  parameters:
  - discreteValueSpec:
      values:
      - 10.0
      - 20.0
    parameterId: iterations
  - doubleValueSpec:
      maxValue: 0.1
      minValue: 0.0001
    parameterId: regularization
    scaleType: UNIT_LINEAR_SCALE
trialJobSpec:
  workerPoolSpecs:
  - containerSpec:
      args:
      - --job_dir=$JOB_DIR
      - --training_dataset_path=$TRAINING_FILE_PATH
      - --validation_dataset_path=$VALIDATION_FILE_PATH
      - --hptune
      imageUri: gcr.io/qwiklabs-asl-04-5e165f533cac/trainer_image
    diskSpec:
      bootDiskSizeGb: 100
      bootDiskType: pd-ssd
    machineS

### Retrieve HP-tuning results.

After the job completes you can review the results using GCP Console or programmatically using the following functions (note that this code supposes that the metrics that the hyperparameter tuning engine optimizes is maximized): 

In [39]:
def get_trials(job_name):
    jobs = aiplatform.HyperparameterTuningJob.list()
    match = [job for job in jobs if job.display_name == JOB_NAME]
    tuning_job = match[0] if match else None
    return tuning_job.trials if tuning_job else None


def get_best_trial(trials):
    metrics = [trial.final_measurement.metrics[0].value for trial in trials]
    best_trial = trials[metrics.index(max(metrics))]
    return best_trial


def retrieve_best_trial_from_job_name(jobname):
    trials = get_trials(jobname)
    best_trial = get_best_trial(trials)
    return best_trial

In [40]:
best_trial = retrieve_best_trial_from_job_name(JOB_NAME)

TypeError: 'NoneType' object is not iterable